In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

np.set_printoptions(precision=4)


In [2]:
data = pd.read_csv('final_df.csv')

In [3]:
data.tail()

,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,nationality,constructor,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,driver_age
25022,2022,4,imola,False,False,False,True,True,latifi,Canadian,williams,19,16,0,0,20,0,0,0,26
25023,2022,4,imola,False,False,False,True,True,mick_schumacher,German,haas,10,17,0,0,17,0,0,0,23
25024,2022,4,imola,False,False,False,True,True,ricciardo,Australian,mclaren,6,18,8,0,11,0,0,0,32
25025,2022,4,imola,False,False,False,True,True,alonso,Spanish,alpine,9,19,2,0,14,0,0,0,40
25026,2022,4,imola,False,False,False,True,True,sainz,Spanish,ferrari,4,20,33,0,3,0,0,0,27


In [4]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()
le5 = LabelEncoder()


In [5]:
data['driver'] = le1.fit_transform(data['driver'])
data['circuit_id'] = le2.fit_transform(data['circuit_id'])
data['nationality'] = le3.fit_transform(data['nationality'])
data['constructor'] = le4.fit_transform(data['constructor'])

In [6]:
df = data.copy()

train = df[df.season <2022]
X_train = train.drop(['podium'], axis = 1)
y_train = train['podium']

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [7]:
appended_data = []
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

params={'fit_intercept': ['True', 'False']}

for fit_intercept in params['fit_intercept']:
    model_params = (fit_intercept)


for i in df[df['season'] == 2022]['round'].unique():

    test = df[(df['season'] == 2022) & (df['round'] == i)]
    X_test = test.drop(['podium'], axis = 1)
    y_test = test['podium']
        

    #scaling
    X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

    # make predictions
    model = LinearRegression(fit_intercept = fit_intercept)
    model.fit(X_train, y_train)
    prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
    prediction_df['podium'] = y_test.reset_index(drop = True)
    prediction_df.sort_values('podium', ascending = True, inplace = True)
    prediction_df['actual'] = prediction_df['podium'].map(lambda x: 1 if x == 1 else 0)
    X_test = pd.DataFrame(scaler.inverse_transform(X_test), columns = X_test.columns)
    X_test['circuit_id'] = X_test['circuit_id'].fillna(0).astype(np.int64, errors='ignore')
    X_test['driver'] = X_test['driver'].fillna(0).astype(np.int64, errors='ignore') 
    prediction_df['driver'] = le1.inverse_transform(X_test['driver'])
    prediction_df['circuit_id'] = le2.inverse_transform(X_test['circuit_id'])
    rank=prediction_df.drop(['results'],axis=1) 
        
    prediction_df.sort_values('results', ascending = True, inplace = True)
    prediction_df.reset_index(inplace = True, drop = True)
    prediction_df['predicted'] = prediction_df.index
    prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x==0 else 0)
        
        
    dff=prediction_df.drop(['actual','podium','circuit_id'],axis=1)
        
        
        
    final= pd.merge(rank,dff,on='driver')
        
    final.sort_values('results', ascending = True, inplace = True)
    final=final[['results','driver','predicted','actual','podium','circuit_id']]
    final=final.reset_index()
    final=final.drop(['index'],axis=1)
    appended_data.append(final)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
appended_data=appended_data.drop(['results','predicted','actual'],axis=1)
appended_data['predicted']=appended_data.index
appended_data=appended_data[['driver','predicted','podium','circuit_id']]
appended_data.predicted += 1
appended_data

,driver,predicted,podium,circuit_id
0,bottas,1,6,bahrain
1,hamilton,2,3,bahrain
2,leclerc,3,1,bahrain
3,alonso,4,9,bahrain
4,kevin_magnussen,5,5,bahrain
...,...,...,...,...
15,mick_schumacher,16,17,imola
16,tsunoda,17,7,imola
17,gasly,18,12,imola
18,latifi,19,16,imola


In [8]:
import pickle
appended_data.to_pickle('appended_data.pkl') # save
appended_data = pd.read_pickle('appended_data.pkl') # load